# Decision Boundary
The main analysis notebook for this experiment. Note that the individual results are NOT included in the repo. We've only included the average over the 50 versions for space reasoning. The granular results can be generated using `main_log_probs.py`. This notebook combines the individual results into an average.

NOTE: This code is legacy and likely won't work out of the box.

In [6]:
import pickle, gzip  
import vllm
import math
import sys
from collections import Counter
sys.path.insert(0, "../../src")
sys.path.insert(0, "../..")
from config import REPO_ROOT
import tqdm
from utils import postprocessing
import os
import json
import copy
import numpy as np
import json
import numpy as np
from pathlib import Path    

In [38]:
probs = []

# load them in from the directory (need to generate t
for i in range(1,11):
    fname = REPO_ROOT / f"analysis/decision_boundary_consistency/income_{i}.logprobs.pkl"  
    with gzip.open(fname, "rb") as f:    
        probs.append(pickle.load(f))

# extract full probs
full_probs = {}

# load the results and store in a dictionary... list
results_list = []
result = postprocessing(REPO_ROOT / 'results/income/llama3_3_70B.json', force=True, distance_metric="gower", save=True, explicit_dataset="")


for ix, item in enumerate(probs):

    probabilities = {}

    for i in tqdm.tqdm(range(len(item))):
        try:
            above = math.exp(item[i][4][59907].logprob)
        except:
            above = 0
        try:
            below = math.exp(item[i][4][39314].logprob)
        except:
            below=0
        probabilities.update({i:{"Below":below, "Above":above}})

    full_probs.update({ix:probabilities})
    results_list.append(copy.deepcopy(result))

# iterate through each results and append answers
updated_list = []
for ix, result_ in enumerate(results_list):
    for k,v in result_.items():
        # if k==139:
        #     print(float(full_probs[ix][k]['Below']))
        v['Below'] = float(full_probs[ix][k]['Below'])
        v['Above'] = float(full_probs[ix][k]['Above'])
    updated_list.append(result_)

averages_above = []
averages_below = []
# for all keys in 1 of the dicts
for k,v in updated_list[0].items():
    aboves = []
    belows = []
    for result in updated_list:
        aboves.append(result[k]['Above'])
        belows.append(result[k]['Below'])
    averages_above.append(float(np.mean(aboves)))
    averages_below.append(float(np.mean(belows)))

    # get the averate

    # set it to the average

# set these valiues
result_edited = postprocessing(REPO_ROOT / 'results/income/llama3_3_70B.json', force=True, distance_metric="gower", save=True, explicit_dataset="")

for k,v in result_edited.items():
    v['Above']=averages_above[k]
    v['Below']=averages_below[k]

# save to json
with open(REPO_ROOT / "results/income/average_two_ways.json", "w") as f:
    json.dump(result_edited, f, indent=2)

Extracted Dataset Name: folktexts_toy_minimal


100%|██████████| 1920/1920 [00:00<00:00, 525759.85it/s]


In [4]:
root = Path(REPO_ROOT / "results/income/llama3_3_70B")
n_files    = 50      
n_examples = 1920        

prediction2num = {'Below $50,000': 0, 'Above $50,000': 1}

pred_matrix = np.zeros((n_examples, n_files), dtype=int)


for col in range(n_files):
    json_file = root / f"income}.json"
    with open(json_file, "r") as f:
        data = json.load(f)

    for k, v in data.items():
        idx = int(k)                     
        if 0 <= idx < n_examples:        
            pred_matrix[idx, col] = prediction2num[v["original_answer"]]
        else:
            raise IndexError(
                f"{json_file}: index {idx} out of range 0–{n_examples-1}"
            )


mean = np.mean(pred_matrix, axis=1)

In [9]:
json_path = REPO_ROOT / "results/income/llama3_3_70B.json"
with open(json_path, "r") as f:
    data = json.load(f)